## **Supreme Court Transcripts Database Design**

### Contents:
 1. Finding Justices Present
 2. Date, Year
 3. Appearances
 4. Sentiment Analysis 
 5. Building the DataFrame
 
## Note to Everyone: Make sure you've uploaded the textfiles from this folder: https://drive.google.com/drive/folders/1aepNIVRUS0rwu-m_fqK7KnWEqhyJFBvW my directory might be different from yours so make sure to check !

In [1]:
import pandas as pd
import regex as re
import numpy as np
import os
from os import listdir
from os.path import isfile,join

# Read in a plain text file
files = []
path = r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles\\"
for i in os.listdir(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles"):
    if i.endswith('.txt'):
        text = open(path+i,encoding="utf8").read()
        files.append(text)

In [2]:
len(files)

1302

In [3]:
cleaned = []
for txt in files:
    clean = re.sub('\xad','',txt) # honestly not sure why this exists but ok
    clean = re.sub('\n','',clean) # no new lines
    clean = re.sub('’',"\\'",clean) #remove weird apostrophe and replace with normal apostrophe
    clean = re.sub('\\\\','',clean)
    cleaned += [clean]

In [4]:
## FIXING SPELLING ERRORS
for transcript in cleaned:
    transcript = re.sub('JUSTICE SC[A-Z]+:','JUSTICE SCALIA:',transcript)
    transcript = re.sub('JUSTICE KEN[A-Z]+','JUSTICE KENNEDY:',transcript)
    transcript = re.sub('JUSTICE [A-Z]+OUT[A-Z]+:','JUSTICE SOUTER:',transcript)
    transcript = re.sub('JUSTICE BRE[A-Z]+:','JUSTICE BREYER:',transcript)
    transcript = re.sub('JUSTICE GIN[A-Z]+:','JUSTICE GINSBURG',transcript)
    transcript = re.sub('JUSTICE [A-Z]+QUIST:','JUSTICE REHNQUIST:',transcript)
    transcript = re.sub('JUSTICE SOTO[A-Z]+:','JUSTICE SOTOMAYOR:',transcript)
    transcript = re.sub('JUSTICE ROB[A-Z]+:','JUSTICE ROBERTS:',transcript)
    transcript = re.sub('JUSTICE STEV[A-Z]+:','JUSTICE STEVENS:',transcript)

### **Finding Justices Present**

In [5]:
def unique(lst):
    uni = []
    for i in lst:
        if i not in uni:
            uni += [i]
    return uni

def case_no(txt):
    return unique(re.findall('No.\s*\d+[-]*\d+',txt))

In [6]:
def justices(txt):
    j = re.findall('JUSTICE[A-Z\s]+:',txt)
    justice = sorted(unique(j))
    cleaned_list = [justice[i][:-1] for i in range(len(justice))]
    return cleaned_list

In [7]:
justices(cleaned[0])

['JUSTICE ALITO',
 'JUSTICE BREYER',
 'JUSTICE GINSBURG',
 'JUSTICE KENNEDY',
 'JUSTICE ROBERTS',
 'JUSTICE SCALIA',
 'JUSTICE SOUTER',
 'JUSTICE STEVENS']

In [8]:
#def justices(texts):
 #   d = {}
  #  for txt in texts:
   #     num = case_no(txt)[0]
    #    j = re.findall('JUSTICE[A-Z\s]+:',txt)
     #   justice = sorted(unique(j))
      #  cleaned_list = [justice[i][:-1] for i in range(len(justice))]
       # d[num] = cleaned_list
    #return d

## **Date, Year**

In [9]:
def date(text):
    return re.findall('\W\w+,*\s+\d+\w*\w*,\s+\d{4}',text)

In [10]:
def year(text):
    return re.findall('\d\d\d\d',date(text))

## **Appearances**

In [11]:
def appearances(text):
    app = re.findall('APPEARANCES:[\s\S]*?Reporting',text)[0]
    app = re.findall('[\s\S]*?;[\s\S]*?\.',app)
    app = [re.sub('\d','',app[i]) for i in range(len(app))]
    remove_appearance = re.sub('APPEARANCES:\s','',app[0])
    app[0] = remove_appearance
    app = [re.sub('[\s\s]+',' ',app[i]) for i in range(len(app))]
    return app

In [12]:
#for i in cleaned:
 #   app = appearances(i)
  #  for j in app:
   #     print(j)
    #print('-----------------------')

## **Sentiment Analysis**

**Checkpoint 10/22**

* need to work on finding the regex pattern between speakers
* General pattern: "SPEAKER: anything they say until the next SPEAKER:"

### STEPS:
1. Extract the sentences from each speaker.
2. Develop a function (actually there's one written in the github link below that you could model yours from; it's very good)
3. Test it on various transcripts to ensure it's generalized

### Note to Amal: Please look at these slides for performing sentiment analysis. They are from my IEOR class, and these techniques are very useful. Please let me know if you would like to go over it together.

https://github.com/ikhlaqsidhu/data-x/blob/master/07a-tools-nlp-sentiment_add_missing_si/notebook-nlp-sentiment-analysis-imdb-afo_v2.ipynb

## OTHER NOTES: you can't use the function below on cases where the justices' names are replaced with "QUESTION"

In [16]:
reporting_pattern = '[A-Z][a-z]+\sReporting'
REPORTING_pattern = '[A-Z]+\sREPORTING'
contents_pattern = 'CONTENTS'
CONTENTS_pattern = 'C O N T E N T S'

In [17]:
## I was trying to extract the speakers from each text. 
all_justices =['JUSTICE GORSUCH','JUSTICE PHILLIPS','JUSTICE SCALIA','JUSTICE ROBERTS','JUSTICE KAVANAUGH',
               'JUSTICE GINSBURG','JUSTICE BREYER','JUSTICE KENNEDY','JUSTICE ALITO','JUSTICE STEVENS',
               'JUSTICE KAGAN','JUSTICE REHNQUIST','JUSTICE THOMAS','JUSTICE SOTOMAYOR','JUSTICE SOUTER','JUSTICE BENNETT']

def dialogue(text,person):
    pattern = re.escape(person)+r":[\s\S]+?[A-Z]+\s*?:"
    sents = re.findall(pattern,text) #regex pattern to find all instances
    sents = [re.sub('\d','',i) for i in sents] # cleaning transcript
    sents = [re.sub('[\s\s]+',' ',i) for i in sents] #cleaning transcript
    sents = [re.sub('Company Official',' ',i) for i in sents if len(re.sub('Company Official',' ',i)) != 0]
    for i in range(len(sents)):
        report1 = re.findall(reporting_pattern,sents[i])
        report2 = re.findall(REPORTING_pattern,sents[i])
        content1 = re.findall(contents_pattern,sents[i])
        content2 = re.findall(CONTENTS_pattern,sents[i])
        if len(report1) > 0:
            sents[i]=re.sub(report1[0],'',sents[i])
        elif len(report2) > 0:
            sents[i]=re.sub(report2[0],'',sents[i])
        elif len(content1) > 0:
            sents[i]=re.sub(content1[0],'',sents[i])
        elif len(content2) > 0:
            sents[i]=re.sub(content2[0],'',sents[i])
        else:
            continue
    sents = [re.sub('Official - Subject to Final Review','',i) for i in sents if len(re.sub('Official - Subject to Final Review','',i))!= 0]
    sents = [re.sub('- Subject to Final Review','',i) for i in sents]
    sents = [re.sub('JUSTICE [A-Z]+:','',i) for i in sents]
    sents = [re.sub('CHIEF|Page|JUSTICE','',i) for i in sents]
    sents = [re.sub('th Street, NW Suite  Company Washington, DC --FOR-DEPO','',i) for i in sents]
    sents = [re.sub('--FOR-DEPO','',i) for i in sents]
    sents = [re.sub('[A-Z\s*\.*]+:','',i) for i in sents]
    return sents


In [18]:
dialogue(cleaned[1123],'REHNQUIST')

[" We'll hear argument next in No. -, TRW, Inc. v. Adelaide Andrews. Mr. Nager"]

In [19]:
case_1 = []
speech = []
for j in all_justices:
    for i in range(100):
        text =  dialogue(cleaned[i],j)
        speech += [text]
        

In [20]:
len(all_justices)

16

In [21]:
speech

[[],
 [" Counsel, are you suggesting you're not going to seek an injunction?",
  " But -- but it's your intention to seek an injunction on the basis of your -",
  ' Counsel -- counsel, the question is -',
  ' -- if I might. If I might',
  ' You intend to seek an injunction on the basis of the order presently before us?',
  ' Okay',
  ' Well -  ',
  ' -- what -- what are we -- sorry.  ',
  " Counsel, what do we do -- I'm sorry.  ",
  " All right. You -- a lot of complaints have to do with vote dilution, and that's what you've been focused on in a number of districts. But what do we do about House District , for example, where, when the legislature sought to take into consideration some concerns along those lines, it -- it then gets attacked from the other direction as -- as discriminating on the basis of race, in violation of the Fourteenth Amendment. How is it -- how is a state supposed to balance its Section obligations against the Fourteenth Amendment obligations? It seems like you'r

# **Building the DataFrame**

#### Justices!
`#print(cleaned[4])`

If we run the above ...

Notice that instead of having the justices' name, it just says "QUESTION". This is usually the case for when it appears there is only one justice present. It seems to be only for Justice Rehnquist.

In [22]:
## Testing do not use
trial = [justices(i) for i in cleaned]
every_justice = []
for i in trial:
    every_justice += i
uniques = unique(every_justice)
for i in range(len(uniques)):
    uniques[i] = re.sub('JUSTICE','',uniques[i])
    uniques[i] = re.sub('\s+','',uniques[i])
#unique(uniques)

In [23]:
justice = [justices(i) for i in cleaned]
for lst in justice:
    for word in range(len(lst)):
        #lst[word] = re.sub('JUSTICE','',lst[word])
        #lst[word] = re.sub('\s+','',lst[word])
        if 'SC' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE SCALIA',lst[word])
        if 'KEN' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE KENNEDY',lst[word])
        if 'TER' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE SOUTER',lst[word])
        if 'BRE' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE BREYER',lst[word])
        if 'GIN' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE GINSBURG',lst[word])
        if 'QUIST' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE REHNQUIST',lst[word])
        if 'SOTO' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE SOTOMAYOR',lst[word])
        if 'ROB' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE ROBERTS',lst[word])
        if 'STEVE' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE STEVENS',lst[word])

In [24]:
#for i in range(len(justice)):
 #   justice[i] = re.sub('JUSTICE','',justice[i])

#### Dates!

In [25]:
date(cleaned[0])[0]

' December 1, 2008'

In [26]:
dates = []
for i in cleaned:
    dates += [date(i)[0]]

In [27]:
len(dates)

1302

#### Year!

In [28]:
years = []
for i in dates:
    years += re.findall('\d\d\d\d',i)

In [29]:
len(years)

1302

#### Case Numbers!

In [30]:
case_no(cleaned[0])

['No. 07-581']

In [31]:
cases = []
for i in cleaned:
    cases += [case_no(i)[0]]

In [32]:
len(cleaned)

1302

#### Appearances!

In [33]:
# don't use this one
def appearances(text):
    app = re.findall('APPEARANCES:[\s\S]*?Reporting.*?',text)
    if len(app) == 0:
        app = re.findall('APPEARANCES:[\s\S]*?REPORTING.*?',text)
        if len(app) == 0:
            app = re.findall('APPEARANCES:[\s\S]*?C O N T E N T S.*?',text)
            if len(app) == 0:
                app = re.findall('APPEARANCES:[\s\S]*?CONTENTS.*?',text)
    app = app[0]
    app = re.findall('[\s\S]*?;[\s\S]*?\.',app)
    app = [re.sub('\d','',app[i]) for i in range(len(app))]
    remove_appearance = re.sub('APPEARANCES:\s','',app[0])
    app[0] = remove_appearance
    app = [re.sub('[\s\s]+',' ',app[i]) for i in range(len(app))]
    return app

In [34]:
def diff(text):
    app = re.findall('APPEARANCES:*[\s\S]*?Reporting.*?',text)
    if len(app) == 0 or len(app[0])>1000:
        app = re.findall('APPEARANCES:[\s\S]*?REPORTING.*?',text)
        if len(app) == 0 or len(app[0])>1000:
            app = re.findall('APPEARANCES:[\s\S]*?C O N T E N T S.*?',text)
            if len(app) == 0 or len(app[0])>1000:
                app = re.findall('APPEARANCES:[\s\S]*?CONTENTS.*?',text)
                if len(app) == 0 or len(app[0])>1000:
                    app = re.findall('APPEARANCES:[\s\S]*?P R O C E E D I N G S.*?',text)
    app = app[0]
    app = re.sub('\d','',app)
    remove_appearance = re.sub('APPEARANCES:*\s','',app)
    app = re.sub('[\s\s]+',' ',remove_appearance)
    return app

In [35]:
ugh = []
c = 1
for i in range(len(cleaned)):
    #print(c)
    try:
        ugh+=[diff(cleaned[i])]
        #print('')
    except:
        ugh += [i]
        continue
    c+=1

In [36]:
for i in ugh:
    if type(i)==int:
        print(i)

1023


In [37]:
ugh[1023]

1023

In [38]:
for i in range(len(ugh)):
    if i == 1023:
        ugh[i] = 1023
    else:
        ugh[i] = re.sub('in No. -','',ugh[i])

In [39]:
ugh[0]

'PAUL SALVATORE, ESQ., New York, N.Y.; on behalf of the Petitioners. DAVID C. FREDERICK, ESQ., Washington, D.C.; on behalf of the Respondents. CURTIS E. GANNON, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting the Respondents. Alderson Reporting'

In [40]:
re.findall(reporting_pattern,cleaned[0])[0]

'Alderson Reporting'

In [41]:
re.sub(re.findall(reporting_pattern,ugh[0])[0],'',ugh[0])

'PAUL SALVATORE, ESQ., New York, N.Y.; on behalf of the Petitioners. DAVID C. FREDERICK, ESQ., Washington, D.C.; on behalf of the Respondents. CURTIS E. GANNON, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting the Respondents. '

In [42]:
for i in range(len(ugh)):
    if i == 1023:
        ugh[i] == 'JASON D. HAWKINS, ESQ., Assistant Federal Public Defender, Dallas, Texas; for Petitioner. WILLIAM M. JAY, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for  Respondent, in support of Petitioner. EVAN A. YOUNG, ESQ., Austin, Texas; for amicus curiae, in support of the judgment below; appointed by this Court.'
    else:
        report1 = re.findall(reporting_pattern,ugh[i])
        report2 = re.findall(REPORTING_pattern,ugh[i])
        content1 = re.findall(contents_pattern,ugh[i])
        content2 = re.findall(CONTENTS_pattern,ugh[i])
        if len(report1) > 0:
            ugh[i]=re.sub(report1[0],'',ugh[i])
        elif len(report2) > 0:
            ugh[i]=re.sub(report2[0],'',ugh[i])
        elif len(content1) > 0:
            ugh[i]=re.sub(content1[0],'',ugh[i])
        else:
            ugh[i]=re.sub(content2[0],'',ugh[i])

In [43]:
bad = 'JASON D. HAWKINS, ESQ., Assistant Federal Public Defender, Dallas, Texas; for Petitioner. WILLIAM M. JAY, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for  Respondent, in support of Petitioner. EVAN A. YOUNG, ESQ., Austin, Texas; for amicus curiae, in support of the judgment below; appointed by this Court.'

In [44]:
no_55 = re.sub("\d", "",cleaned[55])
no_295 = re.sub("\d","",cleaned[295])
app_55 = "THOMAS C. GOLDSTEIN, ESQ., Washington, D.C.; on behalf of Petitioners. DONALD B. VERRILLI, JR., ESQ., Solicitor General, Department of Justice, Washington, D.C.; on behalf of Respondents."
app_295 = "APPEARANCES: Alderson Reporting Company Official - Subject to Final Review TIMOTHY S. BISHOP, ESQ., Chicago, Illinois; on behalf of Petitioners. MALCOLM L. STEWART, ESQ., Deputy Solicitor General, Department of Justice, Washington, D.C.; for United States, as amicus curiae, in support of Petitioners. JEFFREY L. FISHER, ESQ., Stanford, California; on behalf of Respondents."
app_1023 = 'JASON D. HAWKINS, ESQ., Assistant Federal Public Defender, Dallas, Texas; for Petitioner. WILLIAM M. JAY, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for  Respondent, in support of Petitioner. EVAN A. YOUNG, ESQ., Austin, Texas; for amicus curiae, in support of the judgment below; appointed by this Court.'

In [45]:
ugh[55] = app_55
ugh[295] = app_295
ugh[1023] = app_1023

#### People!

In [46]:
people = []
for i in range(len(ugh)):
    p = re.findall('[A-Z]+\s[A-Z\.\s]*[\'*A-Za-z\s*]+',ugh[i])
    people += [p]
    
for i in range(len(people)):
    people[i] = ", ".join(people[i])

In [47]:
anns = []
for i in range(len(people)):
    lst = people[i]
    if "DABNEY" in lst:
        anns += [i]

In [48]:
for i in anns:
    print(data.iloc[i]['Title'])
    print('')

NameError: name 'data' is not defined

In [49]:
data.iloc[1252]['Appearances']

NameError: name 'data' is not defined

In [446]:
#for i in range(len(people)):
 #   need_to_fix = re.findall()

In [447]:
data.iloc[825]

Title                                            Nevada v. Hicks
Case No                                              No. 99-1994
Date                                              March 21, 2001
Year                                                        2001
Justices                                             [REHNQUIST]
Appearances    C. WAYNE HOWLE, ESQ., Carson City, Nevada; on ...
People                WAYNE HOWLE, JAMES ANAYA, BARBARA MCDOWELL
Name: 825, dtype: object

#### Title!

In [50]:
title = [f for f in listdir(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles") if isfile(join(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles", f))]
for i in range(len(title)):
    title[i] = re.sub('.pdf.txt','',title[i])
title.remove('DatabaseDesign1.ipynb')

### ACTUAL DATAFRAME!!!

** Note: Some cases only have one justice, Rehnquist, but this is actually because the rest of the questions appear as "QUESTION:" rather than the justices' name.

In [51]:
#make sure they're all the same length
print(len(justice),
len(title),
len(ugh),
len(people),
len(years),
len(cases))

1302 1302 1302 1302 1302 1302


In [52]:
data = pd.DataFrame({'Title':title,'Case No': cases,'Date':dates,'Year':years,'Justices':justice,'Appearances':ugh,'People':people})#'Justices':justice,'Appearances':people})

In [53]:
data.to_csv(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\data.csv")

## Bills/Acts

In [54]:
import regex as re
import numpy
import os

def unique(texts):
    return list(set(texts))

#return a set of nonunique bill mentioned in the speech.
def bill(texts):
    no_num = re.sub('\d+','',texts)
    no_n = re.sub('\n','',no_num)
    evenly_spaced = re.sub('\s+',' ',no_n)
    b= re.findall("[A-Z][a-z]+\s[A-Z][a-z]+\sAct", evenly_spaced)
    return b
#return a set of unique bill mentioned in the speech.
def unibill(texts):
    return unique(bill(texts))
#return the name and times it appears.
def count(texts):
    bills = bill(texts)
    ret = []
    for i in range(0,len(bills)):
        count = bills.count(bills[i])
        ret.append (str(bills[i]) + " " + ":" + ' '+str(count))
    return unique(ret)


In [55]:
unibill(cleaned[0])
count(cleaned[0])

['Labor Standards Act : 1', 'Labor Relations Act : 2']

In [97]:
bills_mentioned = []
for i in range(len(cleaned)):
    bills_mentioned += [count(cleaned[i])]

In [98]:
len(bills_mentioned)

1302

In [99]:
data['Bills Mentioned'] = bills_mentioned

In [102]:
data['Justices'].iloc[0]

' ALITO,  BREYER,  GINSBURG,  KENNEDY,  ROBERTS,  SCALIA,  SOUTER,  STEVENS'

In [103]:
data['Justices'].iloc[1]

' ALITO,  BREYER,  GINSBURG,  GORSUCH,  KAGAN,  KENNEDY,  ROBERTS,  SOTOMAYOR'

In [104]:
data['Justices'].iloc[2]

' ALITO,  BREYER,  GINSBURG,  ROBERTS,  SCALIA,  SOTOMAYOR'

In [105]:
data['Justices'].iloc[3]

' ALITO,  BREYER,  GINSBURG,  KENNEDY,  ROBERTS,  SCALIA,  STEVENS'

In [77]:
bills_db = data[['Title','Case No','Date','Bills Mentioned']]

In [80]:
#bills_db.to_csv('bills_db.csv')

## Keywords/TF-IDF

In [5]:
import nltk

# nltk.download('all')
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re

from nltk import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet

eng_stopwords = stopwords.words('english')

In [29]:
def text_cleaner(text,stem=False):
    '''
    1. Use regex to remove all special characters (only keep letters)
    2. Make strings to lower case and tokenize / word split reviews
    3. Remove English stopwords
    4. Rejoin to one string
    '''
    #3. Remove punctuation
    text = re.sub("[^a-zA-Z]", " ",text)
    #4. Tokenize into words (all lower case)
    text = text.lower()
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))

    words = word_tokenize(text)
    words = [w for w in words if w not in eng_stopwords]
    return(words)

In [30]:
def term_frequency(document, term):
    words = text_cleaner(document)
    number_of_words = len(words)
    print("Total number of words in document 1 is", number_of_words)

    frequency_of_word = sum([1 for word in words if word.lower() == term])
    print("Frequency of the word", term, "is", frequency_of_word)

    term_frequency = frequency_of_word / number_of_words
    return term_frequency

Total number of words in document 1 is 13697
Frequency of the word Allegations is 17


0.0012411476965758926

In [31]:
text_cleaner(cleaned[0])

['official',
 'subject',
 'final',
 'review',
 'supreme',
 'court',
 'united',
 'states',
 'x',
 'penn',
 'plaza',
 'llc',
 'et',
 'al',
 'petitioners',
 'v',
 'steven',
 'pyett',
 'et',
 'al',
 'x',
 'washington',
 'c',
 'monday',
 'december',
 'entitled',
 'matter',
 'came',
 'oral',
 'argument',
 'supreme',
 'court',
 'united',
 'states',
 'appearances',
 'paul',
 'salvatore',
 'esq',
 'new',
 'york',
 'n',
 'behalf',
 'petitioners',
 'david',
 'c',
 'frederick',
 'esq',
 'washington',
 'c',
 'behalf',
 'respondents',
 'curtis',
 'e',
 'gannon',
 'esq',
 'assistant',
 'solicitor',
 'general',
 'department',
 'justice',
 'washington',
 'c',
 'behalf',
 'united',
 'states',
 'amicus',
 'curiae',
 'supporting',
 'respondents',
 'alderson',
 'reporting',
 'company',
 'official',
 'subject',
 'final',
 'review',
 'c',
 'n',
 'e',
 'n',
 'oral',
 'argument',
 'page',
 'paul',
 'salvatore',
 'esq',
 'behalf',
 'petitioners',
 'david',
 'c',
 'frederick',
 'esq',
 'behalf',
 'respondents',


## SPEECHES DATAFRAME 
### Just justices right now. we can add other people later.

In [56]:
speech = []
for j in all_justices:
    for i in range(1):
        text =  dialogue(cleaned[i],j)
        speech += [text]
        

In [57]:
len(speech)

16

In [58]:
speech_df = pd.DataFrame({'Name':[],'Case':[],'Dialogue':[]})
for i in range(1302):
    title = data['Title'].iloc[i]
    judges = data['Justices'].iloc[i]
    for j in judges:
        words = dialogue(cleaned[i],j)
        words = ".".join(words)
        speech_df = speech_df.append(pd.DataFrame({'Name':[j],'Case':[title],'Dialogue':[words]}))

In [59]:
#print(cleaned[0])

In [60]:
len(speech_df)

7852

In [61]:
#data[['Justices']]

In [63]:
#cleaned[1123]

In [544]:
#speech_df.to_csv('speeched_db.csv')

In [64]:
speech_df.isnull().any()

Name        False
Case        False
Dialogue    False
dtype: bool

## Convert speech to sql



In [69]:
speech_df = speech_df.reset_index().drop('index',axis=1).head()

In [70]:
import sqlite3
con = sqlite3.connect("database.db")

speech_df.to_sql("speech", con, if_exists="replace")


In [71]:
speech_df.head()

,Name,Case,Dialogue
0,JUSTICE ALITO,14 Penn Plaza LLC v. Pyett,Has any court decided this issue of the inter...
1,JUSTICE BREYER,14 Penn Plaza LLC v. Pyett,I ask you -- let me ask you a naive question ...
2,JUSTICE GINSBURG,14 Penn Plaza LLC v. Pyett,But here it wasn't. . Here we are dealing...
3,JUSTICE KENNEDY,14 Penn Plaza LLC v. Pyett,Yes. What -- what - . I don't -- I don't wish...
4,JUSTICE ROBERTS,14 Penn Plaza LLC v. Pyett,"We'll hear argument next in Case -, Penn Plaz..."
